$
\newcommand{\calf}{{\cal F}}
\newcommand{\dnu}{d \nu}
\newcommand{\mf}{{\bf F}}
\newcommand{\vu}{{\bf u}}
\newcommand{\vx}{{\bf x}}
\newcommand{\vy}{{\bf y}}
\newcommand{\vs}{{\bf s}}
\newcommand{\ve}{{\bf e}}
\newcommand{\vd}{{\bf d}}
\newcommand{\mg}{{\bf G}}
\newcommand{\ml}{{\bf L}}
\newcommand{\mg}{{\bf G}}
\newcommand{\mv}{{\bf V}}
\newcommand{\ma}{{\bf A}}
\newcommand{\mi}{{\bf I}}
\newcommand{\mm}{{\bf M}}
\newcommand{\ball}{{\cal B}}
\newcommand{\ptc}{{\Psi TC}}
\newcommand{\diag}{\mbox{diag}}
$

In [25]:
include("fanote_init.jl")

## Section 2.10 Solvers for Chapter 2

Contents for Section 2.10

[Overview](#Overview)

[nsol.jl](#nsol.jl)

- [H-equation Revisited](#H-Equation-revisited)

- [More on the Two-Point BVP](#More-on-the-Two--Point-BVP)

[ptcsol.jl](#ptcsol.jl)

- [More on the Buckling Beam](#Benchmarking-the-Buckling-Beam)

[Projects](#Projects)


### Overview

We will follow the patter of Chapter 1 and present two sovlers, a Newton code and a $\ptc$ code. Both codes are for systems of equations and use direct methods to compute the step. We returned the solution history for the simple two dimensional example in Section 2.6, but will not do that again.

### nsol.jl

__nsol.jl__ solves systems of nonlinear equations and computes the Newton step with direct linear solvers. Let's look at the docstrings.

In [26]:
?nsol

search: nsol nsolsc nsolheq transcode transpose Transpose transpose!



```
nsol(F!, x0, FS, FPS, J!=diffjac!; rtol=1.e-6, atol=1.e-12,
           maxit=20, solver="newton", sham=5, armmax=10, resdec=.1,
           dx = 1.e-7, armfix=false, 
           pdata = nothing, jfact = klfact,
           printerr = true, keepsolhist = false, stagnationok=false)
```

)

C. T. Kelley, 2020

Julia versions of the nonlinear solvers from my SIAM books.  Herewith: nsol

You must allocate storage for the function and Jacobian in advance –> in the calling program <– ie. in FS and FPS

Inputs:

  * F!: function evaluation, the ! indicates that F! overwrites FS, your   preallocated storage for the function.

    So FS=F!(FS,x) or FS=F!(FS,x,pdata) returns FS=F(x)

  * x0: initial iterate

  * FS: Preallcoated storage for function. It is an N x 1 column vector

  * FPS: preallcoated storage for Jacobian. It is an N x N matrix

  * J!: Jacobian evaluation, the ! indicates that J! overwrites FPS, your   preallocated storage for the Jacobian. If you leave this out the   default is a finite difference Jacobian.

    So, FP=J!(FP,FS,x) or FP=J!(FP,FS,x,pdata) returns FP=F'(x). 

    (FP,FS, x) must be the argument list, even if FP does not need FS.   One reason for this is that the finite-difference Jacobian   does and that is the default in the solver.
  * Precision: Lemme tell ya 'bout precision. I designed this code for    full precision   functions and linear algebra in any precision you want. You can declare   FPS as Float64, Float32, or Float16 and nsol will do the right thing if   YOU do not destroy the declaration in your J! function. I'm amazed   that this works so easily. If the Jacobian is reasonably well    conditioned, I can see no reason to do linear algebra in    double precision.

---

Keyword Arguments (kwargs):

  * rtol and atol: relative and absolute error tolerances

  * maxit: limit on nonlinear iterations

solver: default = "newton"

Your choices are "newton" or "chord". However, you have sham at your disposal only if you chose newton. "chord" will keep using the initial derivative until the iterate converges, uses the iteration budget, or the line search fails. It is not the same as sham=Inf, which is smarter.

sham: default = 5 (ie Newton)

This is the Shamanskii method. If sham=1, you have Newton. The iteration updates the derivative every sham iterations. The convergence rate has local q-order sham+1 if you only count iterations where you update the derivative. You need not provide your own derivative function to use this option. sham=Inf is chord only if chord is converging well.

I made sham=1 the default for scalar equations. For systems I'm more aggressive and want to invest as little energy in linear algebra as possible. So the default is sham=5.

armmax: upper bound on stepsize reductions in linesearch

resdec: default = .1

This is the target value for residual reduction. The default value is .1. In the old MATLAB codes it was .5. I only turn Shamanskii on if the residuals are decreasing rapidly, at least a factor of resdec, and the line search is quiescent. If you want to eliminate resdec from the method ( you don't ) then set resdec = 1.0 and you will never hear from it again.

dx: default = 1.e-7

difference increment in finite-difference derivatives       h=dx*norm(x,Inf)+1.e-8

armfix: default = false

The default is a parabolic line search (ie false). Set to true and the stepsize will be fixed at .5. Don't do this unless you are doing experiments for research.

pdata:

precomputed data for the function/Jacobian.  Things will go better if you use this rather than hide the data  in global variables within the module for your function/Jacobian

jfact: default = klfact (tries to figure out best choice) 

If your Jacobian has any special structure, please set jfact to the correct choice for a factorization.

I use jfact when I call PrepareJac! to evaluate the Jacobian (using your J!) and factor it. The default is to use klfact (an internal function) to do something reasonable. For general matrices, klfact picks lu! to compute an LU factorization and share storage with the Jacobian.  You may change LU to something else by, for example, setting jfact = cholseky! if your Jacobian is spd.

klfact knows about banded matrices and picks qr. You should, however RTFM, allocate the extra two upper bands, and use jfact=qr! to override klfact.

If you give me something that klfact does not know how to dispatch on, then nothing happens. I just return the original Jacobian matrix and  nsol will use backslash to compute the Newton step. I know that this is probably not optimal in your situation, so it is  good to pick something else, like jfact = lu.

Please do not mess with the line that calls PrepareJac!. 

```
    FPF = PrepareJac!(FPS, FS, x, ItRules)
```

FPF is not the same as FPS (the storage you allocate for the Jacobian) for a reason. FPF and FPS do not have the same type, even though they share storage. So, FPS=PrepareJac!(FPS, FS, ...) will break things.

printerr: default = true

I print a helpful message when the solver fails. To supress that message set printerr to false.

keepsolhist: default = false

Set this to true to get the history of the iteration in the output tuple. This is on by default for scalar equations and off for systems. Only turn it on if you have use for the data, which can get REALLY LARGE.

stagnationok: default = false

Set this to true if you want to disable the line search and either observe divergence or stagnation. This is only useful for research or writing a book.

Output:

A named tuple (solution, functionval, history, stats, idid,                errcode, solhist) where

solution = converged result functionval = F(solution) history = the vector of residual norms (||F(x)||) for the iteration stats = named tuple of the history of (ifun, ijac, iarm), the number of functions/derivatives/steplength reductions at each iteration.

I do not count the function values for a finite-difference derivative because they count toward a Jacobian evaluation. 

idid=true if the iteration succeeded and false if not.

errcode = 0 if if the iteration succeeded         = -1 if the initial iterate satisifies the termination criteria         = 10 if no convergence after maxit iterations         = 1  if the line search failed

solhist:

This is the entire history of the iteration if you've set keepsolhist=true

solhist is an N x K array where N is the length of x and K is the number of iteration + 1. So, for scalar equations, it's a row vector.

---

# Examples

#### World's easiest problem example.

```jldoctest
 julia> function f!(fv,x)
       fv[1]=x[1] + sin(x[2])
       fv[2]=cos(x[1]+x[2])
       end
f (generic function with 1 method)

julia> x=ones(2,); fv=zeros(2,); jv=zeros(2,2);
julia> nout=nsol(f!,x,fv,jv; sham=1);
julia> nout.history
5-element Array{Float64,1}:
 1.88791e+00
 2.43119e-01
 1.19231e-02
 1.03266e-05
 1.46416e-11

julia> nout.solution
2-element Array{Float64,1}:
 -7.39085e-01
  2.30988e+00

```

#### H-equation example. I'm taking the sham=5 default here, so the convergence is not quadratic. The good news is that we evaluate the Jacobian only once.

```jldoctest
julia> n=16; x0=ones(n,); FV=ones(n,); JV=ones(n,n);
julia> hdata=heqinit(x0, .5);
julia> hout=nsol(heqf!,x0,FV,JV;pdata=hdata);
julia> hout.history
4-element Array{Float64,1}:
 6.17376e-01
 3.17810e-03
 2.75227e-05
 2.35817e-07
```


#### Input

The calling sequence for the Newton solvers in this book are similar, differing mostly in the management of the linear solver and memory allocation. The calling sequence for __nsold.jl__ is

```julia
function nsold(
    F!,
    x0,
    FS,
    FPS,
    J! = diffjac!;
    rtol = 1.e-6,
    atol = 1.e-12,
    maxit = 20,
    solver = "newton",
    sham = 1,
    armmax = 10,
    resdec = 0.1,
    dx = 1.e-7,
    armfix = false,
    pdata = nothing,
    jfact = lu!,
    printerr = true,
    keepsolhist = false,
    stagnationok = false,
)
```

As we said earlier in the chapter, the calling sequence has some new things which are not in __nsolsc.jl__. The most significant are the arrays __FS__ and __FPS__, which preallocate
storage for the function and Jacobian. As we have pointed out earlier, the farther upstream one allocates memory the better, so __nsold.jl__ insists that you allocate an vector __FS__ of the same size as the initial iterate and a matrix 
__FPS__ for the Jacobian. 

The other major new feature is the keyword argument __pdata__. This is the data structure for you to store any precomuted or preallocated data your function evaluation needs. You will almost surely need __pdata__ for any but the most trivial problems. The H-equation example from section 2.6 uses __pdata__ in a serious manner.

You may dimension __x0__ either as $(N,1)$ or $(N,)$, but you must be consistent and dimension __FS__ the same way. __nsold.jl__ expects vectors to be in double precision (Float64). 

The __!__ in the function evaluation __F!__ is to indicate that __nsold__ expects __F__  to overwrite its imput. So,
the way to call __F!__ is to preallocate the storage for the function value in an array __FS__ and then call the function as
```Julia
F!(FS,x)
```
or
```Julia
F!(FS,x,pdata)
```
__nsold.jl__ will figure out if you have populated __pdata__ or left it alone as the default value of __nothing__.


And now for the Jacobian. __nsold.jl__ uses direct methods for linear algebra. If your matrix is dense, the default is to use Julia's __lu!__ function to do an LU factorization. If your matrix is symmetric or symmetric positive definite you can use the __factorization__ keyword to change __lu!__ to __ldlt!__ or __cholesky!__ for example. __nsold.jl__ assumes that the factorization you ask for will overwrite the matrix. Hence, the __factorize__ function in Julia is not what you want for this application.

You will also need to preallocate storage for the Jacobian in the array __FPS__. You may use any legal real precision for __FPS__. Float64 is the default. If you use Float32 you cut the storage for the matrix and the time for the factorization in half. We recommend that you do this if your Jacobian is dense. If you are using the __Sparsesuite__ sparse solvers, then you must store the Jacobian in double precision. __Sparsesuite__ does not support lower precision.

Your Jacobian computation __J!__ must also overwrite it's input. The call looks like
```julia
J!(FV,FP,x)
```
or 
```julia
J!(FV,FP,x,pdata)

```
returns FP=F'(x). The input FP=F(x), which __nsold.jl__ has already compouted, has to be there.


####  H-Equation revisited

In this section we do several experiments to illustrage the advantages of infrequent evaluation and factorization of the Jacobian and mixed precision computation. We will begin with a function that will support our testing. We want to investigate combinations of Newton's method/Shamanskii with ```sham=5``` (the default), storing and factoring the Jacobian in double/single precision, and analytic/forward difference Jacobians. To make this easy we write a function that solves the H-equation with __nsol.jl__ and lets us vary these cases. 

The functions for the residual __heqf!.jl__, the Jacobian __heqJ!.jl__, and the precomputed data 
__heqinit.jl__ are in the large file 
[src/TestProblems/Systems/Hequation.jl](https://github.com/ctkelley/SIAMFANLEquations.jl/blob/master/src/TestProblems/Systems/Hequation.jl)
in the 
[SIAMFANLEquations.jl](https://github.com/ctkelley/SIAMFANLEquations.jl)
repository. 

I'm passing the precomputed data to the function rather than computing it within. This keeps the cost of the precomputed data out of the benchmarking I'll do later.

I'm also using __splat__. I populate a named tuple ```bargs``` to keep the keyword arguments in a convenient place and then, when it's time to give it to __nsol.jl__, the call looks like ```bargs...```. The three dots are the __splat__ and tell __nsol__ to expand bargs and harvest the keyword arguments. 

In [27]:
function htest(x0, FS, FPS, hdata; analytic=false, hsham=5)
    n=length(FS)
    #
    # I've preallocaed x0, FS, and FPS. But they may have been changed by previous runs.
    # The cost of resetting their entries to 1.0 is insignificant. 
    #
    FS.=1.0
    FPS.=1.0
    bargs=(atol = 1.e-10, rtol = 1.e-10, sham = hsham, resdec = .1, pdata=hdata)
    if analytic
        nout=nsol( heqf!, x0, FS, FPS, heqJ!; bargs...)
    else
        nout=nsol( heqf!, x0, FS, FPS; bargs...)
    end
    return nout
end

htest (generic function with 1 method)

To begin we will compare the iteration histories for four cases. We will consider analytic and forward difference Jacobians with the storage and factorization of the Jacobian done in double and single precision. __Theorem 1.2__ says the results should be almost indistinguishable. We will begin with Newton's method.

All we need to do to store and factor Jacobians is to allocate the storage in single precision. That allocation is the line ```FPS32=ones(Float32,n,n);```. Note that we must reset ```FS``` and ```FPS``` after each call to __nsol.jl__ because the solver uses the storage for residuals and Jacobians for the entire iteration. We do this with __broadcast__ after the initial allocaton ```.=1.0``` instead of ```=ones(n,n)``` to avoid reallocation of the Jacobian.

We will print all the residual histories in an array. The history vectors are the same length and are very hard to tell apart until the residual norm is one iteration from stagnation. This is just what the theory predicts. The theory 
(see <cite data-cite="ctk:sirev20"><a href="siamfa.html#ctk:sirev20">(Kel20a)</cite> ) also predicts that there will be little difference between double precision linear algebra and single precistion. We observe this as well.

In [28]:
n=1024; FS=ones(n,); FPS=ones(n,n); FPS32=ones(Float32,n,n); x0=ones(n,); c=.5; hdata = heqinit(x0, c);
nouta64=htest(x0, FS, FPS, hdata;analytic=true, hsham=1);
nouta32=htest(x0, FS, FPS32, hdata; analytic=true, hsham=1);
noutfd64=htest(x0, FS, FPS, hdata; analytic=true, hsham=1);
noutfd32=htest(x0, FS, FPS32,hdata; analytic=false, hsham=1);
[nouta64.history nouta32.history noutfd64.history noutfd32.history]

4×4 Array{Float64,2}:
 4.94274e+00  4.94274e+00  4.94274e+00  4.94274e+00
 2.54130e-02  2.54129e-02  2.54130e-02  2.54118e-02
 4.96643e-07  4.95531e-07  4.96643e-07  4.76209e-07
 9.45712e-15  3.93220e-14  9.45712e-15  1.00644e-12

We will now do the same thing with the default setting of ```sham=5```. The theory correctly predicts that we will see slower convergence. We will be using BenchmarkTools to compare the costs later.

In [29]:
nouta64=htest(x0, FS, FPS, hdata; analytic=true);
noutfd64=htest(x0, FS, FPS, hdata; analytic=false);
nouta32=htest(x0, FS, FPS32, hdata; analytic=true);
noutfd32=htest(x0, FS, FPS32, hdata; analytic=false);
[nouta64.history nouta32.history noutfd64.history noutfd32.history]

6×4 Array{Float64,2}:
 4.94274e+00  4.94274e+00  4.94274e+00  4.94274e+00
 2.54130e-02  2.54129e-02  2.54119e-02  2.54118e-02
 2.19929e-04  2.19928e-04  2.19912e-04  2.19911e-04
 1.88328e-06  1.88327e-06  1.88308e-06  1.88307e-06
 1.61169e-08  1.61167e-08  1.61147e-08  1.61146e-08
 1.37912e-10  1.37909e-10  1.37901e-10  1.37900e-10

This is interesting. We need more iterations, but we evaluate the Jacobian only once for Shamanskii. We can see this by looking at the ```stats``` field of the output tuple. They are all the same, so we will use ```nouta64```.

In [30]:
nouta64.stats

(ifun = [1, 1, 1, 1, 1, 1], ijac = [0, 1, 0, 0, 0, 0], iarm = [0, 0, 0, 0, 0, 0])

The interpretation is that we do a function evaluation at all iterations and a single Jacobian evaluation to compute the $\vx_1$. We do no Jacobian work after that. The default in __nsol.jl__ is to reevaluate the Jacobian if the reduction in the residual norm larger than ```resdec = .1```. You can change ```resdec``` in the keywork arguments.

We will use the [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl) package to look at performance. 
The ```@btime``` command will show compute time and memory allocations for an average of several runs. The averaging will mitigage the effects of the compile time for the first run. 

To begin, we will compare the four versions of Newton's method.


In [31]:
println("analytic, double"); @btime htest($x0, $FS, $FPS, hdata; analytic=true, hsham=1);
println("finite difference, double"); @btime htest($x0, $FS, $FPS, hdata; analytic=false, hsham=1);
println("analytic, single"); @btime htest($x0, $FS, $FPS32, hdata; analytic=true, hsham=1);
println("finite difference, single"); @btime htest($x0, $FS, $FPS32, hdata; analytic=false, hsham=1);

analytic, double
  26.489 ms (6198 allocations: 468.92 KiB)
finite difference, double
  114.909 ms (9264 allocations: 924.27 KiB)
analytic, single
  16.772 ms (6198 allocations: 468.92 KiB)
finite difference, single
  105.954 ms (9264 allocations: 924.27 KiB)


Even for a problem of only modest size, the differences between the analytic Jacobian and the forward difference are significant. The differences between single and double precision linear algebra are less so, but would become more dramatic as the dimension of the problem grows. 

Next, we look at the default ```sham=5``` from __nsol.jl__.

In [32]:
println("analytic, double"); @btime htest($x0, $FS, $FPS, hdata; analytic=true);
println("finite difference, double"); @btime htest($x0, $FS, $FPS, hdata; analytic=false);
println("analytic, single"); @btime htest($x0, $FS, $FPS32, hdata; analytic=true);
println("finite difference, single"); @btime htest($x0, $FS, $FPS32, hdata; analytic=false);

analytic, double
  10.631 ms (2098 allocations: 244.58 KiB)
finite difference, double
  40.140 ms (3120 allocations: 396.36 KiB)
analytic, single
  7.023 ms (2098 allocations: 244.58 KiB)
finite difference, single
  36.127 ms (3120 allocations: 396.36 KiB)


The moral here is pretty clear. We see that compute time is cut by a factor of at least two in all cases. The result for an analytic Jacobian with linear algebra in single precision and ```sham=5``` is 17 times faster than our slowest computation (Newton + finite difference Jacobian + double precision linear algebra). So, do less linear algebra and do it in single precision.

Finally, we will increase the dimenson. As we do that the computation becomes more burdensome, so we will only do two cases, both with an analytic Jacobian.

In [33]:
n=2048; FS=ones(n,); FPS=ones(n,n); FPS32=ones(Float32,n,n); x0=ones(n,); c=.5; hdata = heqinit(x0, c);
println("analytic, double, Newton"); @btime htest($x0, $FS, $FPS, hdata; analytic=true, hsham=1);
println("analytic, single, sham=5"); @btime htest($x0, $FS, $FPS32, hdata; analytic=true);

analytic, double, Newton
  144.587 ms (12342 allocations: 932.92 KiB)
analytic, single, sham=5
  33.991 ms (4146 allocations: 484.58 KiB)


For sufficiently large dimension, the linear algebra cost will dominate the computation. Time should increase by roughly a factor of 8 as the dimension doubles. We are not in that regime yet. One of the projects at the end of this chapter challenges you to get there. Remember that we allocated storage for the Jacobian when we defined ```FPS```, so @btime is not measuring the allocation for the Jacobian.

#### More on the Two-Point BVP

If the Jacobian is sparse and you use the solvers from __SuiteSparse__ then you cannot use single preciscion. Even if the structure of the Jacobian allows you to use the LAPACK solvers or a special-purpose package, there is less benefit in using single precision for linear algebra than in the dense case. We will explore that for the boundary value problem, where we can use [BandedMatrices.jl](https://github.com/JuliaMatrices/BandedMatrices.jl) and __qr!__ for the linear solver.

The function 

### ptcsol.jl

__ptcsol.jl__ is our $\ptc$ solve. As usual, we begin with the docstrings ...

In [34]:
?ptcsol

search: ptcsol ptcsolsc PartialQuickSort



ptcsol(F!, x0, FS, FPS, J! = diffjac!; rtol=1.e-6, atol=1.e-12,                maxit=20, dt0=1.e-6, dx=1.e-7, pdata = nothing, jfact = klfact,                printerr = true, keepsolhist = false)

C. T. Kelley, 2020

Julia versions of the nonlinear solvers from my SIAM books.  Herewith: some new stuff ==> ptcsol

You must allocate storage for the function and Jacobian in advance –> in the calling program <– ie. in FS and FPS

Inputs:

  * F!: function evaluation, the ! indicates that F! overwrites FS, your   preallocated storage for the function.

    So, FV=F!(FV,x) or FV=F!(FV,x,pdata) returns FV=F(x)
  * x0: initial iterate

  * FS: Preallcoated storage for function. It is an N x 1 column vector.

You may dimension it as (n,) or (n,1). (n,) is best, but the solvers can deal with it either way.

  * FPS: preallcoated storage for Jacobian. It is an N x N matrix

  * J!: Jacobian evaluation, the ! indicates that J! overwrites FPS, your   preallocated storage for the Jacobian. If you leave this out the   default is a finite difference Jacobian.

    So, FP=J!(FP,FV,x) or FP=J!(FP,FV,x,pdata) returns FP=F'(x);   (FP,FV, x) must be the argument list, even if FP does not need FV.   One reason for this is that the finite-difference Jacobian   does and that is the default in the solver.
  * Precision: Lemme tell ya 'bout precision. I designed this code for    full precision   functions and linear algebra in any precision you want. You can declare   FPS as Float64, Float32, or Float16 and ptcsol will do the right thing if   YOU do not destroy the declaration in your J! function. I'm amazed   that this works so easily. If the Jacobian is reasonably well    conditioned, I can see no reason to do linear algebra in    double precision.

---

Keyword Arguments (kwargs):

rtol and atol: relative and absolute error tolerances

dt0: initial time step. The default value of 1.e-3 is a bit conservative and is one option you really should play with. Look at the example where I set it to 1.0!

maxit: limit on nonlinear iterations, default=100. 

This is coupled to dt0. If your choice of dt0 is too small (conservative) then you'll need many iterations to converge and will need a larger value of maxit

For PTC you'll need more iterations than for a stright-up nonlinear solve. This is part of the price for finding the  stable solution.

dx: default = 1.e-7

difference increment in finite-difference derivatives       h=dx*norm(x)+1.e-6

pdata:

precomputed data for the function/Jacobian.  Things will go better if you use this rather than hide the data  in global variables within the module for your function/Jacobian

jfact: default = klfact (tries to figure out best choice) 

If your Jacobian has any special structure, please set jfact to the correct choice for a factorization.

I use jfact when I call PTCUpdate to evaluate the Jacobian (using your J!) and factor it. The default is to use klfact (an internal function) to do something reasonable. For general matrices, klfact picks lu! to compute an LU factorization and share storage with the Jacobian.  You may change LU to something else by, for example, setting jfact = cholseky! if your Jacobian is spd.

klfact knows about banded matrices and picks qr. You should, however RTFM, allocate the extra two upper bands, and use jfact=qr! to override klfact.

If you give me something that klfact does not know how to dispatch on, then nothing happens. I just return the original Jacobian matrix and  ptcsol will use backslash to compute the Newton step.

I know that this is probably not optimal in your situation, so it is  good to pick something else, like jfact = lu.

printerr: default = true

I print a helpful message when the solver fails. To supress that message set printerr to false.

keepsolhist: default = false

Set this to true to get the history of the iteration in the output tuple. This is on by default for scalar equations and off for systems. Only turn it on if you have use for the data, which can get REALLY LARGE.

Output:

A named tuple (solution, functionval, history, stats, idid,                errcode, solhist) where

solution = converged result functionval = F(solution) history = the vector of residual norms (||F(x)||) for the iteration stats = named tuple of the history of (ifun, ijac, iarm), the number of functions/derivatives/steplength reductions at each iteration.

I do not count the function values for a finite-difference derivative because they count toward a Jacobian evaluation. 

idid=true if the iteration succeeded and false if not.

errcode = 0 if if the iteration succeeded         = -1 if the initial iterate satisifies the termination criteria         = 10 if no convergence after maxit iterations         = 1  if the line search failed

solhist:

This is the entire history of the iteration if you've set keepsolhist=true

solhist is an N x K array where N is the length of x and K is the number of iteration + 1. So, for scalar equations, it's a row vector.


#### More on the Buckling Beam

## Projects

### More on the H-equation

- Run the H-equation tests with $c=.99$ and larger dimensions ($2^k$ for $k=12, 13, 14$). Are the values of ```sham``` and ```resdec``` optimal for this case? How, for example, is ```resdec = Inf``` (your author's favorite value). To see why I like ```sham=Inf``` have a look at
<cite data-cite="brent"><a href="siamfa.html#brent">(Bre73)</cite>.

- What happens with $c=1$? Is the chord method a good idea? 
Don't look at
<cite data-cite="ctk:sirev20"><a href="siamfa.html#ctk:sirev20">(Kel20a)</cite>,
<cite data-cite="ctk:n1"><a href="siamfa.html#ctk:n1">(DK80)</cite>, or
<cite data-cite="ctk:chord"><a href="siamfa.html#ctk:chord">(DK83)</cite> before trying to figure things out on your own.

- Increase the dimension as much as you can and compare single and double precision linear algebra.
    
- Duplicate the table on page 125 of <cite data-cite="chand"><a href="siamfa.html#chand">(Cha60)</cite>.    


### Automatic Differentiation

__(not trivial!)__ Try using __automatic differention__ to compute the Jacobian for the examples in this chapter and use the results to write a Jacobian evaluation function to pass to __nsol.jl__.
You will first have to decide how you want to do that. Two of mature packages are [ForwardDiff.jl](#https://github.com/JuliaDiff/ForwardDiff.jl)
<cite data-cite="forwarddiff"><a href="siamfa.html#forwarddiff">(RPL16)</cite>
and [Zygote.jl](#https://github.com/FluxML/Zygote.jl) 
<cite data-cite="zygote"><a href="siamfa.html#zygote">(CoRR)</cite>.
Books like
<cite data-cite="grautodiff"><a href="siamfa.html#grautodiff">(Gri00)</cite> explain the
algorithmic differences between these packages.
    
How does the perfomance compate to an analytic or forward difference Jacobian? 

### Nested Iteration

Solving a differential or integral equation
by __nested iteration__ or __grid sequencing__ means resolving
the rough features of the solution of a differential or
integral equation on a coarse mesh, interpolating the
solution to a finer mesh, resolving on the finer mesh, and
then repeating the process until you have a solution on a target
mesh.

Apply this idea to some of the examples in the text, using piecewise
linear interpolation to move from coarse to fine meshes. If the
discretization is second-order accurate and you halve the mesh
width at each level, how should you terminate the solver at
each level? What kind of iteration statistics would tell you that
you've done a satisfactory job?


### Sparse Differencing

Write Jacobian evaluation functions using sparse differencing for the boundary value problem and the buckling beam examples. You can use the literature, for exampe the methods from 
<cite data-cite="curtispr"><a href="siamfa.html#curtisor">(CPR74)</cite> or
<cite data-cite="colmore"><a href="siamfa.html#colmore">(CM83)</cite>.  <cite data-cite="ctk:newton"><a href="siamfa.html#ctk:newton">(Kel03)</cite> has a Matlab code for banded matrices that, while convertable to Julila, needs some work to explicltly store the Jacobian as a BandedMatrix so
[BandedMatrices.jl](https://github.com/JuliaMatrices/BandedMatrices.jl) will solve the linear system efficiently.
Another alternative is to use __sparsefdiff__ 
<cite data-cite="sparsediff"><a href="siamfa.html#sparsediff">(RMGH20)</cite> from
[SparseDiffTools.jl](#https://github.com/JuliaDiff/SparseDiffTools.jl). 
    
No matter what you do, you'll need to think about fill-in, symbolic factorization in the general case, and storage. 
    
Have fun!    